# Reconstruct Trajectory - New Approach

Since previous approaches haven't been successful, let's try something different. It turned out that it is not possible to eliminate the influence of gravity only by knowing its initial direction. This is far too inaccurate, hence there is a big drift if you try to calculate the position.

The following idea came up: We can assume that after one repetition of an exercise the sensor is in the same position as in the beginning. Hence, it should be possible first to calculate the position without eliminating the influence of gravity and afterwards we can look at the resulting position, which gives us the direction of gravity. Moreover, we can also quantitative determine the influence of gravity, because we know the distance traveled as well as the time.

In [2]:
import skinematics as skin
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from scipy.signal import savgol_filter
import functionsMasterProjectMeinhart as fmpm

In [3]:
%matplotlib auto

Using matplotlib backend: TkAgg


In [17]:
def calc_trajectory_orientation(acc_lin_g, vel_ang_degps, sampling_rate=256):
    '''
    This function calculates the trajectory by means of a set of linear acceleration and angular velocity data.
    The sampling rate has to be given in Hz.
    Calculate the position in an upright frame (z-axis aligned with gravity).
    
    Inputs
    ------
    acc_lin_g: linear acceleration [g]
    
    vel_ang_degps: angular velocity [deg/s]
    
    sampling_rate: sampling rate of the measured signals in Hz
    
    
    Returns
    -------
    Dictionary with upright position, velocity and orientation (vector-part of quaternion)
    '''
    
    
    g_mps2 = 9.80665 # [m/s^2]
    
    acc_lin = np.array(acc_lin_g) * g_mps2 # linear acceleration in m/s^2
    
    vel_ang = np.array(vel_ang_degps) * np.pi/180 # angular velocity in rad/s
    
    delta_t = 1/sampling_rate
    


    acc_upright_gravity = np.array([0, 0, g_mps2]) # gravity in upright position [m/s^2]
    
    # check number of data points
    if acc_lin.ndim is 1 or vel_ang.ndim is 1:
        num_steps = 1
    elif np.shape(acc_lin)[0] <= np.shape(vel_ang)[0]:
        num_steps = np.shape(acc_lin)[0]
    else:
        num_steps = np.shape(vel_ang)[0]
    
    # reserve memory for position and orientation vectors
    #    --> one step more, because of initial condition
    pos_init_space = np.zeros((num_steps+1, 3))
    vel_init_space = np.zeros((num_steps+1, 3))
    q_vec = np.zeros((num_steps+1, 3)) # vector part of quaternion
         
    
    for ii in range(num_steps):
        
        omega_abs = np.linalg.norm(vel_ang[ii])
        
        # avoid division by zero (or value close to zero)
        if omega_abs < 0.00001 and omega_abs > -0.00001:
            q_vec[ii+1] = q_vec[ii]
        
        else:
            delta_q_vec = vel_ang[ii]/omega_abs * np.sin(omega_abs*delta_t/2)
            q_vec[ii+1] = skin.quat.q_mult(q_vec[ii], delta_q_vec)
        
        R_init_space_obj = skin.quat.convert(q_vec[ii+1] , to='rotmat')
        
        acc_init_space = np.dot(R_init_space_obj, acc_lin[ii])
        
        vel_init_space[ii+1] = vel_init_space[ii] + acc_init_space*delta_t

        pos_init_space[ii+1] = pos_init_space[ii] +  vel_init_space[ii]*delta_t + 0.5*acc_init_space*delta_t**2
    
    
    # bring velocity and position in an upright frame:
    
    acc_gravity = np.array(pos_init_space[-1]) *2 / (delta_t*num_steps)**2
    
    acc_gravity_abs = np.linalg.norm(acc_gravity)
    acc_gravity_norm = np.array(acc_gravity) / acc_gravity_abs # normalize

    vec_upright = np.array([0,0,1])

    v = np.cross(acc_gravity_norm, vec_upright)
    s = np.linalg.norm(v) # sine of angle
    c = np.dot(acc_gravity_norm, vec_upright) # cosine of angle

    # skew-symmetric cross-product matrix of v
    Vx = np.array([[ 0,   -v[2], v[1] ],
                   [ v[2], 0 ,  -v[0] ],
                   [-v[1], v[0], 0    ]])

    R_upright_init = np.identity(3) + Vx + np.dot(Vx,Vx) * (1-c)/s**2
    
    # rotate velocity and position profile into upright frame
    vel_upright_gravity = np.array([np.dot(R_upright_init, vel_init_space[ii]) for ii in range(num_steps+1)])
    pos_upright_gravity = np.array([np.dot(R_upright_init, pos_init_space[ii]) for ii in range(num_steps+1)])
    
    
    # eliminate influence of gravity:
    
    vel_upright = np.zeros((num_steps+1, 3))
    pos_upright = np.zeros((num_steps+1, 3))
    
    for ii in np.arange(num_steps+1):
        vel_upright[ii] = vel_upright_gravity[ii] - np.array([0,0,acc_gravity_abs])*(delta_t*ii)
        pos_upright[ii] = pos_upright_gravity[ii] - 0.5*np.array([0,0,acc_gravity_abs])*(delta_t*ii)**2
    
      
    data_dict = {}
    data_dict['pos'] = pos_upright
    data_dict['vel'] = vel_upright
    data_dict['q_vec'] = q_vec
    data_dict['R_upright_init'] = R_upright_init
    
    return data_dict

In [5]:
np.zeros(3)

array([0., 0., 0.])

In [6]:
'''start_min = 2
start_sec = 20

stop_min = 2
stop_sec = 23.8'''

sampling_rate = 256 # [Hz]

start_min = 2
start_sec = 20

stop_min = 2
stop_sec = 37.1

start_time = start_min*60 + start_sec # [s]
stop_time = stop_min*60 + stop_sec # [s]

sensor_data = fmpm.get_sensor_data(in_file='Subject_01/subject01.csv',
                                   start_time=start_time,
                                   stop_time=stop_time)

plt.figure()
fmpm.plot_signal(sensor_data['Acc'],sensor_data['time'],g_to_ms2=True)

In [7]:
window_length = 81
polyorder = 3

gyr_filt = savgol_filter(sensor_data['Gyr'],window_length, polyorder, axis=0)
plt.figure()
fmpm.plot_signal(gyr_filt,sensor_data['time'],
            Title='Angular Velocity Profile (filtered)',
            xLabel=r'$time \enspace [s]$',
            yLabel=r'$vel \enspace [\frac{deg}{s}]$')

In [10]:
window_length = 81
polyorder = 3

acc_filt = savgol_filter(sensor_data['Acc'],window_length, polyorder, axis=0)
plt.figure()
fmpm.plot_signal(acc_filt,sensor_data['time'],
            Title='Accelereation Profile (filtered) --> select time range',
            g_to_ms2=False)

selected_points = plt.ginput(2)
start_time_sel_index = int((selected_points[0][0]-start_time)*sampling_rate)
stop_time_sel_index  = int((selected_points[1][0]-start_time)*sampling_rate)

E:\Programme\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\matplotlib\backend_bases.py:2453: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [11]:
acc_filt_selected = acc_filt[start_time_sel_index:stop_time_sel_index,:]
gyr_filt_selected = gyr_filt[start_time_sel_index:stop_time_sel_index,:]
time_selected = sensor_data['time'][start_time_sel_index:stop_time_sel_index]

In [12]:
np.shape(time_selected)

(747,)

In [13]:
plt.figure()
fmpm.plot_signal(acc_filt_selected,time_selected,
            Title='Selected Accelereation Profile (filtered)',
            g_to_ms2=False)

In [18]:
traj_data_filt = calc_trajectory_orientation(acc_filt_selected,
                                             gyr_filt_selected,
                                             sampling_rate=sampling_rate)

In [19]:
traj_data_filt['pos']

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.66885333e-07,  5.62358305e-06, -2.33150201e-07],
       [ 3.46479403e-06,  2.24607836e-05, -8.66578159e-07],
       ...,
       [-2.83432839e-04, -2.52616750e-05,  4.10665700e-05],
       [-1.44683259e-04, -1.74185314e-05,  1.89531318e-05],
       [-3.55271368e-15,  8.88178420e-16,  0.00000000e+00]])

In [29]:
# get position values in x,y,z-direction
[pos_x, pos_y, pos_z] = [traj_data_filt['pos'][:,ii] for ii in range(3)]

# generate vectors for initial sensor position
scale_arrow = 0.2 # scale arroe length
[init_x, init_y, init_z] = [traj_data_filt['R_upright_init'][:,ii] for ii in range(3)]
[init_x, init_y, init_z]= np.array([init_x, init_y, init_z]) * scale_arrow
origin = [0,0,0]
X, Y, Z = zip(origin,origin,origin) 
U, V, W = zip(init_x, init_y, init_z)

# define section for plotting (if scale = 1 --> x,y,z-limits = 1 [m])
scale_plot_section = 0.7

fig = plt.figure(figsize=(8,8))
ax = fig.gca(projection='3d')
ax.plot(pos_x, pos_y, pos_z)
ax.set_title('Calculated Trajectory',fontsize='13',fontweight='bold')
ax.set_xlim(np.array([-1,1])*scale_plot_section)
ax.set_ylim(np.array([-1,1])*scale_plot_section)
ax.set_zlim(np.array([-1,1])*scale_plot_section)
ax.set_xlabel(r'$x \enspace [m]$')
ax.set_ylabel(r'$y \enspace [m]$')
ax.set_zlabel(r'$z \enspace [m]$')
ax.set_aspect('equal')

# plot initial sensor orientation
ax.quiver(X,Y,Z,U,V,W,arrow_length_ratio=0.2,color='r')

# add text for axis labels of initial orientation
text_dis_scale = 1.1 # to ensure that labels and arrows do not overlap
for label, pos in zip(['x','y','z'], np.array([init_x, init_y, init_z])*text_dis_scale):
    ax.text(pos[0], pos[1], pos[2], label, color='red', fontsize='11')

plt.show()

In [ ]:
acc_gravity = np.array(traj_data_filt['pos'][-1]) *2 / (stop_time-start_time)**2 
np.linalg.norm(acc_gravity)

In [ ]:
# initial acceleration shall be only caused by gravity
acc_gravity_abs = np.linalg.norm(acc_gravity)
acc_gravity_norm = np.array(acc_gravity) / acc_gravity_abs # normalize

a = acc_gravity_norm
b = np.array([0,0,1])

v = np.cross(a, b)
s = np.linalg.norm(v) # sine of angle
c = np.dot(a, b) # cosine of angle

# skew-symmetric cross-product matrix of v
Vx = np.array([[ 0,   -v[2], v[1] ],
               [ v[2], 0 ,  -v[0] ],
               [-v[1], v[0], 0    ]])

R_upright_init = np.identity(3) + Vx + np.dot(Vx,Vx) * (1-c)/s**2

pos_upright = np.array(np.dot(R_upright_init, acc_gravity))

In [ ]:
pos_upright

Suppose you want to find a rotation matrix R that rotates unit vector a onto unit vector b.

In [ ]:
a = np.array([1/np.sqrt(2), 1/np.sqrt(2), 0])
b = np.array([0,0,1])

v = np.cross(a, b)
s = np.linalg.norm(v) # sine of angle
c = np.dot(a, b) # cosine of angle

# skew-symmetric cross-product matrix of v
Vx = np.array([[ 0,   -v[2], v[1] ],
               [ v[2], 0 ,  -v[0] ],
               [-v[1], v[0], 0    ]])

R = np.identity(3) + Vx + np.dot(Vx,Vx) * (1-c)/s**2

In [ ]:
np.dot(R, a)

In [ ]:
np.rad2deg(np.arcsin(s))

In [ ]:
np.rad2deg(np.arccos(c))

In [ ]:
Vx

In [ ]:
R

$9.81\frac{m}{s^2}$

The next idea is to use an upright space as reference frame. This should be possible because we assume that the sensor is not moved or rotated in the beginning. Therefore we know the direction of the gravity vector in the beginning. Hence, we can calculate a transformation matrix for an upright position, even if we only know the z-axis of the upright frame. Consequently, the new reference frame's z-component is independent from the starting position, but the alignment of the other two components (x,y) depend on the starting position.

In order to clarify this, let's assume we measure the following acceleration values in the beginning (when the sensor is not moved or rotated):

$$\vec{acc}^{initial}_{gravity} = \begin{pmatrix} 2.5 \\ 3.3 \\ 8.9 \end{pmatrix}\frac{m}{s^2}$$

This yields and absolute value of approximately $9.81\frac{m}{s^2}$.

Now we want to find the rotation matrix $\textbf{R}_{upright}^{initial}$, which allows us to align both z-axes of the two frames.

By using rotations around x-axis and y-axis, we have to define now the sequence of the two rotations. So let's say first we want to rotate the initial frame (sensor) with respect to it's x-axis and afterwards we rotate around the body-fixed y-axis. This gives us the following equation: 

$\textbf{R}_{x}(\alpha) \cdot \textbf{R}_{y}(\beta) = \textbf{R}_{upright}^{initial}$

We already know the z-component of $\textbf{R}_{upright}^{initial}$, which is $$\frac{1}{\begin{vmatrix}\vec{acc}^{initial}_{gravity}\end{vmatrix}} \cdot \vec{acc}^{initial}_{gravity} = \begin{pmatrix} 0.255 \\ 0.336 \\ 0.907 \end{pmatrix}$$

So let's calculate the two angles $\alpha$ and $\beta$ as well as the corresponding rotation matrix: